In [1]:
### Required Libraries ###
from datetime import datetime
from dateutil.relativedelta import relativedelta
from botocore.vendored import requests

### Functionality Helper Functions ###
def parse_int(n):
    """
    Securely converts a non-integer value to integer.
    """
    try:
        return int(n)
    except ValueError:
        return float("nan")

def build_validation_result(is_valid, violated_slot, message_content):
    """
    Define a result message structured as Lex response.
    """
    if message_content is None:
        return {"isValid": is_valid, "violatedSlot": violated_slot}

    return {
        "isValid": is_valid,
        "violatedSlot": violated_slot,
        "message": {"contentType": "PlainText", "content": message_content},
    }

### Dialog Actions Helper Functions ###
def get_slots(intent_request):
    """
    Fetch all the slots and their values from the current intent.
    """
    return intent_request["currentIntent"]["slots"]


def elicit_slot(session_attributes, intent_name, slots, slot_to_elicit, message):
    """
    Defines an elicit slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "ElicitSlot",
            "intentName": intent_name,
            "slots": slots,
            "slotToElicit": slot_to_elicit,
            "message": message,
        },
    }

def build_response_card(title, subtitle, options):
    """
    Build a responseCard with a title, subtitle, and an optional set of options which should be displayed as buttons.
    """
    buttons = None
    if options is not None:
        buttons = []
        for i in range(min(5, len(options))):
            buttons.append(options[i])

    return {
        'contentType': 'application/vnd.amazonaws.card.generic',
        'version': 1,
        'genericAttachments': [{
            'title': "This is your X_Ray.",
            "imageUrl": "",
            'buttons': [
                    {
                        "text": "Yes",
                        "value": "Yes"
                    },
                    {
                        "text": "No",
                        "value": "No"
                    },
                ]
            }
        ]
    }

def delegate(session_attributes, slots):
    """
    Defines a delegate slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {"type": "Delegate", "slots": slots},
    }


def close(session_attributes, fulfillment_state, message, response_card):
    """
    Defines a close slot type response.
    """
    response = {
        'sessionAttributes': session_attributes,
        'dialogAction': {
            'type': 'Close',
            'fulfillmentState': fulfillment_state,
            'message': message,
            'responseCard': response_card
        }
    }   

    return response

def close_two(session_attributes, fulfillment_state, message):
    """
    Defines a close slot type response.
    """
    response = {
        'sessionAttributes': session_attributes,
        'dialogAction': {
            'type': 'Close',
            'fulfillmentState': fulfillment_state,
            'message': message
        }
    }   

    return response

def recommend_action(slots):
    
    recomendation = "Seek immediate medical attention."
   
    if slots['smoking'] == "Yes":
        recomendation += " Smoking damages your body's natural defenses against the bacteria and viruses that cause pneumonia."
    
    if slots['chronic_disease'] == "Yes":
        recomendation += " You're more likely to get pneumonia if you have asthma, chronic obstructive pulmonary disease (COPD) or heart disease."
    
    if slots['immune_system'] == "Yes":
        recomendation += " People who have HIV/AIDS, who've had an organ transplant, or who receive chemotherapy or long-term steroids are at greater risk."
    
    return recomendation 

### Intents Handlers ###
def medical_recomendation(intent_request):
    """
    Performs dialog management and fulfillment for recommending a portfolio.
    """
    first_name = get_slots(intent_request)["firstName"]
    age = get_slots(intent_request)["age"]
    hospitalization = get_slots(intent_request)["hospitalization"]
    x_rays = get_slots(intent_request)["x_rays"]
    smoking = get_slots(intent_request)["smoking"]
    chronic_disease = get_slots(intent_request)["chronic_disease"]
    immune_system = get_slots(intent_request)["immune_system"]
    coronavirus = get_slots(intent_request)["coronavirus"]
    covid_risk = get_slots(intent_request)["covid_risk"]
    travel = get_slots(intent_request)["travel"]
    source = intent_request["invocationSource"]
    
    if source == "DialogCodeHook":
        # This code performs basic validation on the supplied input slots.
        # Perform basic validation on the supplied input slots.
        # Use the elicitSlot dialog action to re-prompt
        # for the first violation detected.
        
        # Gets all the slots
        slots = get_slots(intent_request)

        # Validates user's input using the validate_data function
        validation_result = validate_data(age)

        # If the data provided by the user is not valid,
        # the elicitSlot dialog action is used to re-prompt for the first violation detected.
        if not validation_result["isValid"]:
            slots[validation_result["violatedSlot"]] = None  # Cleans invalid slot

            # Returns an elicitSlot dialog to request new data for the invalid slot
            return elicit_slot(
                intent_request["sessionAttributes"],
                intent_request["currentIntent"]["name"],
                slots,
                validation_result["violatedSlot"],
                validation_result["message"])
                        
        if hospitalization == "No": 
            slots['x_rays'] = "No"
                
        if x_rays == "Yes":
            
            return close(
                intent_request["sessionAttributes"],
                "Fulfilled",
                {
                    "contentType": "PlainText",
                    "content": """{} thank you for your information;
                    based on your X-Ray imaging results, you do not have pneumonia with a 96.98% confidence level.
                    """.format(
                        first_name, 
                    )
                },
                {
                    "version": '2',
                    "contentType": "application/vnd.amazonaws.card.generic",
                    "genericAttachments": [
                          {
                             "imageUrl":"https://github.com/Luminous171/Project_2/blob/main/Images/Predicted_Images/normal%20-%20NORMAL.jpeg?raw=true",
                          }
                       ] 
                }
            )
        
        if slots['coronavirus' ] == "Yes": 
            slots['covid_risk'] == "No"
            slots['travel'] == "No"
            
        if slots['coronavirus' ] == "Yes" or slots['covid_risk'] == "Yes" or slots['travel'] == "Yes":
            alternative_recommendation = "If you have been diagnosed with COVID - 19 or suspect that you may have COVID - 19 seek immediate medical attention.\
            Severe COVID - 19 can damage lungs and make its victim more likely to get more infections in the lungs.\
            The difference between pneumonia and COVID-19 is usually determined clinically."
            
            return close_two(
                intent_request["sessionAttributes"],
                "Fulfilled",
                {
                    "contentType": "PlainText",
                    "content": """{} thank you for your information;
                    based on the information you provided, my recommendation is to do the following: {}
                    """.format(
                        first_name, alternative_recommendation
                    ),
                },
            )
            
        if slots['smoking'] == "No" and slots['chronic_disease'] == "No" and slots['immune_system'] == 'No':
            alternative_recommendation_two =  "Continue to monitor your symptoms. If they persist for more than two weeks or rapidly worsen make an appointment with your doctor."
            return close_two(
                intent_request["sessionAttributes"],
                "Fulfilled",
                {
                    "contentType": "PlainText",
                    "content": """{} thank you for your information;
                    based on the information you provided, my recommendation is to do the following: {}
                    """.format(
                        first_name, alternative_recommendation_two
                    ),
                },
            )
       
        # Fetch current session attibutes
        output_session_attributes = intent_request["sessionAttributes"]

        return delegate(output_session_attributes, get_slots(intent_request))

    initial_recommendation = recommend_action(get_slots(intent_request))

    return close_two(
        intent_request["sessionAttributes"],
        "Fulfilled",
        {
            "contentType": "PlainText",
            "content": """{} thank you for your information;
            based on the information you provided, my recommendation is to do the following: {}
            """.format(
                first_name, initial_recommendation
            ),
        },
    )

### YOUR DATA VALIDATION CODE STARTS HERE ###
def validate_data(age):

    if age is not None:
        age = parse_int(age)
        if age <18:
            return build_validation_result(
                False,
                "age",
                "You should be at least 18 years old to use this service, "
                "please provide a valid age",
        )
        if age >65:
            return build_validation_result(
                False,
                "age",
                "Adults older than age 65 are at a higher risk of getting pneumonia. Please seek immediate medical attention.")

    # A True results is returned if age or amount are valid
    return build_validation_result(True, None, None)
### YOUR DATA VALIDATION CODE ENDS HERE ###

### Intents Dispatcher ###
def dispatch(intent_request):
    """
    Called when the user specifies an intent for this bot.
    """

    intent_name = intent_request["currentIntent"]["name"]

    # Dispatch to bot's intent handlers
    if intent_name == "MedicalRecommendation":
        return medical_recomendation(intent_request)

    raise Exception("Intent with name " + intent_name + " not supported")

### Main Handler ###
def lambda_handler(event, context):
    """
    Route the incoming request based on intent.
    The JSON body of the request is provided in the event slot.
    """

    return dispatch(event)

ModuleNotFoundError: No module named 'botocore'